This notebook shows how to compute a layernorm forward operation using cuDNN.

$$\text{LayerNorm}(x) = \frac{x-\mu}{\sqrt{\sigma^2 + \epsilon}}\cdot\gamma+\beta$$

Where $\mu = E[x]$ and $\sigma^2 = Var[x]$ are taken over all inputs in a batch.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/cudnn-frontend/blob/main/samples/python/01_matmul_bias.ipynb)

## Prerequisites and Setup
This notebook requires an NVIDIA GPU. If `nvidia-smi` fails, go to Runtime -> Change runtime type -> Hardware accelerator and confirm a GPU is selected.

In [ ]:
# get_ipython().system('nvidia-smi')

If running on Colab, you will need to install the cudnn python interface.

In [ ]:
# get_ipython().system('pip install nvidia-cudnn-cu12')
# get_ipython().system('pip install nvidia-cudnn-frontend')
# get_ipython().system('pip3 install --pre torch --index-url https://download.pytorch.org/whl/nightly/cu128')

#### General Setup
Create a cudnn handle, which is a per device handle used to initialize cudnn context.

In [ ]:
import cudnn
import torch
import sys

torch.manual_seed(1)
handle = cudnn.create_handle()

print("Running with cudnn backend version:", cudnn.backend_version())

assert torch.cuda.is_available()

### LayerNorm Training
Problem Sizes:
- Batch Size: 4
- Sequence Size: 1024
- Embedding Dimension: 768

In [ ]:
batch, seq_size, embedding_dim = 4, 1024, 768

input_type = torch.float16

# Epsilon is a small number to prevent division by 0.
epsilon_value = 1e-3

Create input tensor GPU buffers. We use PyTorch to allocate GPU tensors so we can reuse them easily when we calculate reference outputs.

In [ ]:
# allocate input tensor memory, initialize them to random numbers
x_gpu = torch.randn(
    batch * seq_size,
    embedding_dim,
    1,
    1,
    dtype=input_type,
    requires_grad=True,
    device="cuda",
).to(memory_format=torch.channels_last)
scale_gpu = torch.randn(
    1, embedding_dim, 1, 1, dtype=input_type, requires_grad=True, device="cuda"
).to(memory_format=torch.channels_last)
bias_gpu = torch.randn(
    1, embedding_dim, 1, 1, dtype=input_type, requires_grad=True, device="cuda"
).to(memory_format=torch.channels_last)

# Epsilon must be a scalar value on the cpu.
epsilon_cpu = torch.full(
    (1, 1, 1, 1), epsilon_value, dtype=torch.float32, requires_grad=False, device="cpu"
)

Compute reference ouputs and allocate output tensor GPU buffers

In [ ]:
# Create the reference computation outputs here before the cuDNN computation, in order to use .empty_like() to create our output buffers
out_expected = torch.nn.functional.layer_norm(
    x_gpu,
    [embedding_dim, 1, 1],
    weight=scale_gpu.squeeze(0),
    bias=bias_gpu.squeeze(0),
    eps=epsilon_value,
)

mean_expected = x_gpu.to(torch.float32).mean(dim=(1, 2, 3), keepdim=True)

inv_var_expected = torch.rsqrt(
    torch.var(x_gpu.to(torch.float32), dim=(1, 2, 3), keepdim=True) + epsilon_value
)

# Allocate output tensor memory using PyTorch
# PyTorch has calculated their shapes already, so we can simply use .empty_like()
out_gpu = torch.empty_like(out_expected)
mean_gpu = torch.empty_like(mean_expected)
inv_var_gpu = torch.empty_like(inv_var_expected)

#### Create cuDNN graph


Here we assign UIDs for tensors. UIDs are a unique identifier that will allow us to provide a mapping from tensors created from cuDNN graph api calls, such as `graph.tensor_like()`, to the underlying device memory that will be used to store these tensors. Virtual tensors don't require explicit memory allocated for them, but non-vritual tensors like inputs or outputs will need to have UIDs assigned to them. 

Alternatively, one can use handles directly in the mapping, however using UIDs can be more convinient for caching of cuDNN graphs.

For each of our inputs {X, Scale, Bias, Epsilon} and our outputs {Out, Mean, Inverse Variance}, we allocate a UID. 

In [ ]:
from enum import Enum


class UID(Enum):
    X = 0
    SCALE = 1
    BIAS = 2
    EPSILON = 3
    OUT = 4
    MEAN = 5
    INV_VAR = 6

In [ ]:
# Create the cuDNN graph.
graph = cudnn.pygraph(
    handle=handle,
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
)

# Create tensor handles with the graph API, assign UIDs.
x = graph.tensor_like(x_gpu.detach()).set_name("X").set_uid(UID.X.value)
scale = graph.tensor_like(scale_gpu.detach()).set_name("scale").set_uid(UID.SCALE.value)
bias = graph.tensor_like(bias_gpu.detach()).set_name("bias").set_uid(UID.BIAS.value)
epsilon = graph.tensor_like(epsilon_cpu).set_name("epsilon").set_uid(UID.EPSILON.value)

# Add a layernorm operation
(out, mean, inv_var) = graph.layernorm(
    name="layernorm",
    input=x,
    norm_forward_phase=cudnn.norm_forward_phase.TRAINING,
    scale=scale,
    bias=bias,
    epsilon=epsilon,
)

# Enable all outputs, by default outputs are disabled
out.set_name("output").set_output(True).set_data_type(out_expected.dtype).set_uid(
    UID.OUT.value
)
mean.set_name("mean").set_output(True).set_data_type(mean_expected.dtype).set_uid(
    UID.MEAN.value
)
inv_var.set_name("inv_var").set_output(True).set_data_type(
    inv_var_expected.dtype
).set_uid(UID.INV_VAR.value)

#### Build the graph

In [ ]:
# Build the graph
graph.build([cudnn.heur_mode.A, cudnn.heur_mode.FALLBACK])

# To run this block more than once, we need to re-run the previous block to get a new graph.
# The same instance of a graph can not be built twice.

#### Execute the graph

After validating and building a cuDNN graph,  we can now execute it. To do this, we have to provide input and output buffers. We do this by using the previously allocated UIDs to associate between tensor handles generated from the graph API, and their underlying memory. 

The desired input values need to be stored in these buffers before the `graph.execute` call. Because we have done a reference computation, we can simply reuse the buffers we have allocated via PyTorch.

Note that the EPISLON UID expects a cpu buffer, 

In [ ]:
# Mapping of (UIDs -> memory)
variant_pack = {
    UID.X.value: x_gpu,
    UID.SCALE.value: scale_gpu,
    UID.BIAS.value: bias_gpu,
    UID.EPSILON.value: epsilon_cpu,
    UID.OUT.value: out_gpu,
    UID.MEAN.value: mean_gpu,
    UID.INV_VAR.value: inv_var_gpu,
}

workspace = torch.empty(graph.get_workspace_size(), device="cuda", dtype=torch.uint8)
graph.execute(variant_pack, workspace)

Test cuDNN's output against PyTorch's and check correctness

In [ ]:
torch.cuda.synchronize()

# compare to reference output
torch.testing.assert_close(out_gpu, out_expected, rtol=5e-3, atol=5e-3)
torch.testing.assert_close(inv_var_gpu, inv_var_expected, rtol=5e-3, atol=5e-3)
torch.testing.assert_close(mean_gpu, mean_expected, rtol=5e-3, atol=5e-3)

Perform Cleanup

In [ ]:
cudnn.destroy_handle(handle)